In [ ]:
# you need to install tflearn to the runtime
!pip install tflearn

In [ ]:
# Libraries needed for NLP
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# Libraries needed for Tensorflow processing
import tensorflow as tf
from tensorflow.python.framework import ops # cannot directly call, tf.reset_default_graph()... so I imported this...https://stackoverflow.com/questions/40782271/attributeerror-module-tensorflow-has-no-attribute-reset-default-graph
import numpy as np
import tflearn
import random
import json

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
from google.colab import files
files.upload()

Saving intents.json to intents.json


{'intents.json': b'{"intents": [\r\n        {"tag": "greeting",\r\n         "patterns": ["Hi", "How are you", "Is anyone there?", "Hello", "Good day", "Hey", "Greetings", "Ohoyo", "Hello There", "Um.. Hi There", "Hi there handsome"],\r\n         "responses": ["Hi, There", "Helloooo", "Good Day",  "Hi there how\'s it going?", "Helooo There"],\r\n         "context_set": ""\r\n        },\r\n        {"tag": "goodbye",\r\n         "patterns": ["Bye", "See you later", "Goodbye", "See You", "TTYL", "See you then handsome", "See you then... ttyl"],\r\n         "responses": ["See you later", "Have a nice day", "Bye! Bye.", "Let\'s Talk Soon", "Ciao", "Bye bye.. I\'ll be here waiting for you to come back", "I\'ll miss you"],\r\n         "context_set": ""\r\n        },\r\n        {"tag": "thanks",\r\n         "patterns": ["Thanks", "Thank you", "That\'s helpful", "Thanks alot", "Thnx", "tks", "Thank yooooo"],\r\n         "responses": ["... anytime", "Any time!", "I\'m here to help", "hehe... That

In [ ]:
# import our chat-bot intents file
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [ ]:
intents

{'intents': [{'context_set': '',
   'patterns': ['Hi',
    'How are you',
    'Is anyone there?',
    'Hello',
    'Good day',
    'Hey',
    'Greetings',
    'Ohoyo',
    'Hello There',
    'Um.. Hi There',
    'Hi there handsome'],
   'responses': ['Hi, There',
    'Helloooo',
    'Good Day',
    "Hi there how's it going?",
    'Helooo There'],
   'tag': 'greeting'},
  {'context_set': '',
   'patterns': ['Bye',
    'See you later',
    'Goodbye',
    'See You',
    'TTYL',
    'See you then handsome',
    'See you then... ttyl'],
   'responses': ['See you later',
    'Have a nice day',
    'Bye! Bye.',
    "Let's Talk Soon",
    'Ciao',
    "Bye bye.. I'll be here waiting for you to come back",
    "I'll miss you"],
   'tag': 'goodbye'},
  {'patterns': ['Thanks',
    'Thank you',
    "That's helpful",
    'Thanks alot',
    'Thnx',
    'tks',
    'Thank yooooo'],
   'responses': ['... anytime',
    'Any time!',
    "I'm here to help",
    "hehe... That's what I'm here for",
    'You 

In [ ]:
words = []
classes = []
documents = []
ignore = ['?']
# loop through each sentence in the intent's patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each and every word in the sentence
        w = nltk.word_tokenize(pattern)
        # add word to the words list
        words.extend(w)
        # add word(s) to documents
        documents.append((w, intent['tag']))
        # add tags to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [ ]:
# Perform stemming and lower each word as well as remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore]
words = sorted(list(set(words)))

# remove duplicate classes
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

114 documents
22 classes ['alcoholAbuse', 'botName', 'callEmergencyPanic', 'changeSexualOrientation', 'cheatedOnMarriage', 'cheatedOnPartner', 'cryDuringTherapy', 'dealWithStress', 'emergency', 'findSelf', 'goodbye', 'greeting', 'lostLovedOne', 'panicAttack', 'playBinauralBeats', 'playMusic', 'suicide', 'thanks', 'therapyNervous', 'tooManyIssues', 'usersName', 'wrongWithSummerSchool']
159 unique stemmed words ["'m", "'s", "'ve", ',', '.', '...', 'a', 'abus', 'aft', 'again', 'alcohol', 'alot', 'an', 'and', 'anyon', 'anyth', 'ar', 'attack', 'be', 'beat', 'being', 'bina', 'boy', 'brain', 'breath', 'broth', 'bye', 'ca', 'cal', 'can', 'che', 'commit', 'counsel', 'coupl', 'cry', 'day', 'deal', 'died', 'do', 'drink', 'dur', 'emerg', 'every', 'far', 'fath', 'feel', 'find', 'for', 'friend', 'gay', 'get', 'girl', 'go', 'going', 'gon', 'good', 'goodby', 'grandp', 'greet', 'guil', 'handsom', 'hav', 'hello', 'help', 'hey', 'hi', 'homosex', 'how', 'husband', 'i', 'in', 'into', 'is', 'issu', 'it', 'j

In [ ]:
# create training data
training = []
output = []
# create an empty array for output
output_empty = [0] * len(classes)

# create training set, bag of words for each sentence
for doc in documents:
    # initialize bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stemming each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is '1' for current tag and '0' for rest of other tags
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffling features and turning it into np.array
random.shuffle(training)
training = np.array(training)

# creating training lists
train_x = list(training[:,0])
train_y = list(training[:,1])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [ ]:
# resetting underlying graph data
ops.reset_default_graph()

# Building neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# softmax algorithm
# linear regression

# Defining model and setting up tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')

# Start training
model.fit(train_x, train_y, n_epoch=1000, batch_size=16, show_metric=True)
model.save('model.tflearn')

Training Step: 7999  | total loss: 0.00295 | time: 0.049s
| Adam | epoch: 1000 | loss: 0.00295 - acc: 1.0000 -- iter: 112/114
Training Step: 8000  | total loss: 0.00290 | time: 0.053s
| Adam | epoch: 1000 | loss: 0.00290 - acc: 1.0000 -- iter: 114/114
--
INFO:tensorflow:/content/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [ ]:
import pickle
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )

In [ ]:
# restoring all the data structures
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

In [ ]:
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [ ]:
# load the saved model
model.load('./model.tflearn')

INFO:tensorflow:Restoring parameters from /content/model.tflearn


In [ ]:
def clean_up_sentence(sentence):
    # tokenizing the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stemming each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# returning bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenizing the pattern
    sentence_words = clean_up_sentence(sentence)
    # generating bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [ ]:
ERROR_THRESHOLD = 0.30
def classify(sentence):
    # generate probabilities from the model
    results = model.predict([bow(sentence, words)])[0]
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # a random response from the intent
                    return print(random.choice(i['responses']))

            results.pop(0)

In [ ]:
classify('heeeeeeeeloooooo')

[('panicAttack', 0.9445149)]

In [ ]:
response('What are you hours of operation?')

Hi, There


In [ ]:
response('What is menu for today?')

Today's special is Chicken Tikka


In [ ]:
#Some of other context free responses.
response('Do you accept Credit Card?')

We accept VISA, Mastercard and AMEX


In [ ]:
response('Where can we locate you?')

Our Address is: 1000 Bridge Avenue, London EC3N 4AJ, UK


In [ ]:
response('That is helpful')

Happy to help!


In [ ]:
response('Bye')

Have a nice day


In [ ]:
#Adding some context to the conversation i.e. Contexualization for altering question and intents etc.
# create a data structure to hold user context
context = {}


ERROR_THRESHOLD = 0.75

def classify(sentence):
    # generate probabilities from the model
    results = model.predict([bow(sentence, words)])[0]
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # set context for this intent if necessary
                    if 'context_set' in i:
                        if show_details: print ('context:', i['context_set'])
                        context[userID] = i['context_set']

                    # check if this intent is contextual and applies to this user's conversation
                    if not 'context_filter' in i or \
                        (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                        if show_details: print ('tag:', i['tag'])
                        # a random response from the intent
                        return (random.choice(i['responses']))

            results.pop(0)

In [ ]:
response('Can you please let me know the delivery options?')

'Yes, we provide home delivery through UBER Eats and Zomato?'

In [ ]:
response('What is the menu?')

You can check out the food menu at www.mymenu.com


In [ ]:
context

{'123': 'food'}

In [ ]:
# we will remove the context... # this tag sets the context to empty
response("Hi there!", show_details=True)

context: 
tag: greeting


'Hello, thanks for visiting'

In [ ]:
context

{'123': ''}

In [ ]:
# a menu isn't printed because it is skipped
response('What is the menu?')

In [ ]:
# creating a chatbot function
#  "Shepherd {} is {} years old.".format(shepherd, age)
from builtins import input # I had to reimport input... because it was being overrided somewhere

username = 'John'
# facebook profanity dataset will be used to filter out the bad words
#  words_to_censor = []
# s + s[-1:]
response_to_censored = [
  "heeey!! that's not nice", "Language!! {}".format(username), "You shouldn't say bad words {}".format(username), 
  "That's not very nice", "{}".format(username.upper() + username[-1].upper() + username[-1].upper() + '!!!'),
  "hey hey...", "Please don't go there", "um... {}... keep kalm and don't swear".format(username), "such profanity...", 
  "huuuuuuuuuuuuuuuuhh.... You didn't", "sigh... oh dear {} what am I to do with you?".format(username), "oh boyy!!! we got a tough one"
]
empty_responses = [
  "...", "I'm sorry, I didn't quite understand that", "Didn't quite catch that", "I don't think I follow"
]

def chat():
  print("Hi this is Pikaia. Your companion for a sound mind")
  input_sentence = ''
  pikaiya_response = ''
  exit_commands = ['1','2']  
  while(1):
    try: 
      input_sentence = input('{} :'.format(username))
      if input_sentence.lower() in ['quit', 'q', 'exit', 'close application', 'quit application']:
        print('Pikaia: ttyl {}'.format(username))
        break
      if input_sentence.lower() in words_to_censor:
        print('Pikaia: ',random.choice(response_to_censored))      
          
      else:
        pikaiya_response = response(input_sentence)
        if (pikaiya_response is None) or (str(pikaiya_response).strip()==""):
          print('Pikaia: ',random.choice(empty_responses))
        else: 
          print('Pikaia: ',pikaiya_response)          
    except KeyError:
      print('Pikaia: ','Um... what was that?')
 

In [ ]:
chat()

Hi this is Pikaia. Your companion for a sound mind
John :hi what's your name
Pikaia:  It's <USER> ryt?
John :what's your name?
Pikaia:  It's Pikaia
John :oh ok I'm thoshitha
Pikaia:  Just go! You start with the most prevalent issue by talking with your therapist. Healing one issue can have a ripple effect and heal other issues as well.
John :play music
Pikaia:  Let's have a music therapy session then
John :I'm having a panic attack
Pikaia:  here, do this with me.... breathe in as slowly, deeply and gently as you can, through your nose. then breathe out slowly, deeply and gently through your mouth.... do this a couple of times.. do you feel better now?
John :call someone
Pikaia:  Sit tight... working on it
John :call someone
Pikaia:  On it...
John :hi
Pikaia:  Helloooo
John :how are you
Pikaia:  Good Day
John :call someone
Pikaia:  Ciao
John :q
Pikaia: ttyl John


# |V| disregard the following code blocks

In [ ]:
input = 'QUIT'
if input in ['quit', 'q', 'exit', 'close application', 'close aplication' , 'quit application']:
      print('yea')

In [ ]:
username = 'John'
words_to_censor = ['badword']
# s + s[-1:]
response_to_censored = [
  "heeey!! that's not nice", "Language!! {}".format(username), "You shouldn't say bad words {}".format(username), 
  "That's not very nice", "{}".format(username.upper() + username[-1].upper() + username[-1].upper() + '!!!'),
  "hey hey...", "I don't ", "um... {}... keep kalm and don't swear".format(username), "such profanity...", 
  "huuuuuuuuuuuuuuuuhh.... You didn't", "sigh... oh dear {} what am I to do with you?".format(username), "oh boyy!!! we got a tough one"
]
print(random.choice(response_to_censored))

sigh... oh dear John what am I to do with you?


CSV to JSON converter

In [ ]:
# upload csv file

from google.colab import files
uploaded = files.upload()

Saving toConvert.csv to toConvert.csv


In [ ]:
import pandas as pd
import io
df = pd.read_csv(io.BytesIO(uploaded['toConvert.csv']))     
# df = pd.read_csv(io.BytesIO(uploaded['Terms-to-Block.csv']))     

In [ ]:
import pandas as pd

df.to_json (r'converted.json')

In [ ]:
from google.colab import files
files.download('converted.json') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>